In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import geo

In [48]:
data = pd.read_csv('data_bc.csv')
dataBC = data[data['addressRegion'] == 'BC']
dataBC = dataBC.sort_values(by='addressLocality')
dataBC.head()

C:\Users\bubly\AppData\Local\Temp\ipykernel_28572\191140646.py:1: DtypeWarning: Columns (42,93,259) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_bc.csv')


,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,description,price,priceCurrency,property-beds,...,MLS Area Minor,View,Foundation Details,SqFt Range,Building Area Source,Buyer Agent Designation,Lot Measurement,Lot Size Source,Topography,Current Financing
4337,"6263 MULLIGAN DRIVE, 100 Mile House, BC V0K2E3",100 Mile House,BC,V0K2E3,51.584053,-121.143606,* PREC - Personal Real Estate Corporation. WA...,647000.0,CAD,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4739,"6470 MERCER ROAD, 100 Mile House, BC V0K2E3",100 Mile House,BC,V0K2E3,51.611912,-121.236554,This tidy 2 bed / 2 bath manufactured home is ...,425000.0,CAD,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,"6847 FAWN LK AC01 ROAD, 100 Mile House, BC V0K1X1",100 Mile House,BC,V0K1X1,51.554452,-120.981004,When only the best will do!! Ultimate log home...,1100000.0,CAD,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618,"825 SCOTT ROAD, 100 Mile House, BC V0K2E0",100 Mile House,BC,V0K2E0,51.634147,-121.301281,* PREC - Personal Real Estate Corporation. Me...,559000.0,CAD,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2216,"4741 TELQUA DRIVE, 100 Mile House, BC V0K2Z0",100 Mile House,BC,V0K2Z0,51.747026,-121.347015,* PREC - Personal Real Estate Corporation. No...,399900.0,CAD,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
dataBC = dataBC.filter([
    'streetAddress',
    'addressLocality',
    'addressRegion',
    'postalCode',
    'latitude',
    'longitude',
    'price',
    'property-beds',
    'property-baths',
    'property-sqft',
    'Property Type', #Agricultural, Condo, Condo/Townhouse, Duplex, Manufactured Home, Mobile Home, MultiFamily, Single Family, Townhome, Vacant Land
    'Square Footage',
    ])
data_bc_single_family = dataBC[dataBC['Property Type'] == 'Single Family']

In [50]:
metro_vancouver_cities = [
    "Vancouver", "Burnaby", "Richmond", "Surrey", "Coquitlam",
    "North Vancouver", "West Vancouver", "New Westminster",
    "Delta", "Port Coquitlam", "Port Moody", "Langley"
]
data_bc_single_family = data_bc_single_family[data_bc_single_family['addressLocality'].isin(metro_vancouver_cities)]
sample = data_bc_single_family.head()
sample

,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,price,property-beds,property-baths,property-sqft,Property Type,Square Footage
7409,"51 N SEA AVENUE, Burnaby, BC V5B 1K4",Burnaby,BC,V5B 1K4,49.284733,-122.979824,2188000.0,5.0,3.0,"2,515",Single Family,"2,515 SQFT"
12098,"7815 ALLMAN STREET, Burnaby, BC V5E 2A9",Burnaby,BC,V5E 2A9,49.235055,-122.950665,3999000.0,8.0,9.0,"6,399",Single Family,"6,399 SQFT"
6686,"8274 WEDGEWOOD STREET, Burnaby, BC V3N 1C4",Burnaby,BC,V3N 1C4,49.231928,-122.927040,1868000.0,3.0,1.0,"1,775",Single Family,"1,775 SQFT"
12124,"6555 DENBIGH AVENUE, Burnaby, BC V5H 3R7",Burnaby,BC,V5H 3R7,49.224338,-122.984424,3990000.0,5.0,4.0,"2,736",Single Family,"2,736 SQFT"
4441,"7088 HALIFAX STREET, Burnaby, BC V5A 1L8",Burnaby,BC,V5A 1L8,49.267710,-122.956486,2998888.0,6.0,5.0,"4,946",Single Family,"4,946 SQFT"


In [51]:
# data_bc_single_family.sort_values(by='Price', ascending=False)
data_bc_single_family.to_csv('data_bc_1.csv')

In [52]:
lat_lon_array = sample[['latitude', 'longitude']].to_numpy()
lat_lon_array

array([[  49.2847328, -122.9798237],
       [  49.2350546, -122.950665 ],
       [  49.2319281, -122.9270398],
       [  49.2243379, -122.9844235],
       [  49.2677095, -122.9564857]])

In [53]:
# write code here for using geo.haversine() to get individual address distances

results = []
for lat, lon in lat_lon_array:
    amenities = geo.get_specific_amenities(lat, lon, radius=3000)
    conv_distance = []
    transit_distance = []
    school_distance = []

    if amenities:
        for a in amenities:
            dist = geo.haversine(lat, lon, a['latitude'], a['longitude'])
            if a.get('shop') in ['convencince', 'grocery']:
                conv_distance.append(dist)
            elif a.get('amenity') in ['bus_station', 'subway_station', 'railway_station']:
                transit_distance.append(dist)
            elif a.get('amenity') in ['school', 'university']:
                school_distance.append(dist)
            
    results.append({
        'latitude': lat,
        'longitude': lon,
        'avg_convenience_dist': np.mean(conv_distance),
        'avg_transit_distance': np.mean(transit_distance),
        'avg_school_distance': np.mean(school_distance)
    })

results_df = pd.DataFrame(results)
results_df

c:\Users\bubly\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\bubly\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,latitude,longitude,avg_convenience_dist,avg_transit_distance,avg_school_distance
0,49.284733,-122.979824,NaN,NaN,2.351229
1,49.235055,-122.950665,2.691901,NaN,2.436122
2,49.231928,-122.927040,NaN,2.714781,NaN
3,49.224338,-122.984424,0.250622,NaN,1.973735
4,49.267710,-122.956486,1.324208,NaN,2.681776


In [54]:
sample = sample.merge(
    results_df,
    how='left',
    on=['latitude', 'longitude']
)

In [55]:
sample

,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,price,property-beds,property-baths,property-sqft,Property Type,Square Footage,avg_convenience_dist,avg_transit_distance,avg_school_distance
0,"51 N SEA AVENUE, Burnaby, BC V5B 1K4",Burnaby,BC,V5B 1K4,49.284733,-122.979824,2188000.0,5.0,3.0,"2,515",Single Family,"2,515 SQFT",NaN,NaN,2.351229
1,"7815 ALLMAN STREET, Burnaby, BC V5E 2A9",Burnaby,BC,V5E 2A9,49.235055,-122.950665,3999000.0,8.0,9.0,"6,399",Single Family,"6,399 SQFT",2.691901,NaN,2.436122
2,"8274 WEDGEWOOD STREET, Burnaby, BC V3N 1C4",Burnaby,BC,V3N 1C4,49.231928,-122.927040,1868000.0,3.0,1.0,"1,775",Single Family,"1,775 SQFT",NaN,2.714781,NaN
3,"6555 DENBIGH AVENUE, Burnaby, BC V5H 3R7",Burnaby,BC,V5H 3R7,49.224338,-122.984424,3990000.0,5.0,4.0,"2,736",Single Family,"2,736 SQFT",0.250622,NaN,1.973735
4,"7088 HALIFAX STREET, Burnaby, BC V5A 1L8",Burnaby,BC,V5A 1L8,49.267710,-122.956486,2998888.0,6.0,5.0,"4,946",Single Family,"4,946 SQFT",1.324208,NaN,2.681776


The code below is for normalizing data once we have it all cleaned up. We can ignore it for now and come back to it once we have clean data with all our columns

In [56]:
# features = ['Price', 'Bedrooms', 'Bathrooms', 'Acreage', 'Square Footage']
# scaler = MinMaxScaler()

# data_scaled = sample.copy()
# data_scaled[features] = scaler.fit_transform(sample[features])

# data_scaled["Price"] = 1 - data_scaled['Price']
# data_scaled['Garage'] = data_scaled['Garage'].map({'Yes': 1, 'No': 0})
# data_scaled['Parking'] = data_scaled['Parking'].map({'Yes': 1, 'No': 0})
# data_scaled.head()

In [57]:
# score_features = [
#     'Price',
#     'Bedrooms',
#     'Bathrooms',
#     'Square Footage',
#     'Acreage',
#     'Garage',
#     'Parking'
# ]

# weights = np.array([1/len(score_features)] * len(score_features))
# data_scored = data_scaled.copy()
# data_scored['Score'] = data_scaled[score_features].dot(weights)
# data_scored.sort_values(by='Score', ascending=False, inplace=True)
# data_scored